# Feature Selection & Data Leakage Check## objectives1. **Identify and classify all145feature**2. **detect Data Leakage risk**3. **generatefeatureclassification list**## WARNING: Data Leakage is ？**Data Leakage** is traindatain ** haveat **。### Example:- ** **: Use `total_pymnt`（Total payment） `loan_status`- Problem: have completed Total payment- Result:modelattrain ， feature- ** **: Use `annual_inc`（Annual income）、`dti`（Debt-to-income ratio） `loan_status`- Reason:Information known at loan application time- Result:model---## feature### **PRE-LOAN Features (Can use)**Information known at loan application time:- Borrower basic information- Credit history (before application)- Loan application information### **POST-LOAN Features (Must delete - Leakage)**Information generated after loan issuance:- Payment records- Recovered amountsUnpaid principal### **Metadata Features (to beDelete - no predictive value)**- ID fields- URL- Geographic location details### **Target Variable (Target variable)**- `loan_status` - The variable we want to predict---

In [ ]:
# Cell 1: Import librariesimport pandas as pdimport numpy as npimport warningswarnings.filterwarnings('ignore')pd.set_option('display.max_columns', None)pd.set_option('display.max_rows', 200)print(" Libraries imported successfully")

In [ ]:
# Cell 2: Load data import os # Priority load loan_with_desc.csv，If not exists, load loan.csv if os.path.exists('../../data/loan_with_desc.csv'): print("load loan_with_desc.csv...") df = pd.read_csv('../../data/loan_with_desc.csv', low_memory=False) print(f" Loaded subset of data with desc") else: print("load loan.csv...") df = pd.read_csv('../../data/loan.csv', low_memory=False) # havedesc data df = df[df['desc'].notna() & (df['desc'].astype(str).str.strip().str.len() > 1)].copy() print(f" Filtered data with desc") print(f"\nData shape: {df.shape[0]:,} rows × {df.shape[1]} columns") print(f"\nallcolumns :") for i, col in enumerate(df.columns, 1): print(f"{i:3d}. {col}")

---## featureat allfeature rowsdetailed classification。 **temporal order**：- Information known before loan application ➜ **PRE-LOAN**- Information generated after loan issuance ➜ **POST-LOAN**---

In [ ]:
# Cell 3: Define feature classificationprint("=" * 80)print("Feature Classification Definition")print("=" * 80)# ========================================================================# PRE-LOAN FEATURES (Pre-loan - Can use)# ========================================================================pre_loan_features = [# 1. Basic loan application information'loan_amnt', # Loan amount'funded_amnt', # Funded amount'funded_amnt_inv', # Funded amount'term', # Term'int_rate', # Interest rate'installment', # Installment'grade', # Grade'sub_grade', # Grade'purpose', # Loan purpose'title', # Loan title'desc', # Loan description（For OCEAN feature extraction）# 2. Borrower basic information'emp_title', # Employer job title'emp_length', # Employment length'home_ownership', # Home ownership status'annual_inc', # Annual income'verification_status', # Verification status'issue_d', # Loan issuance date（Available, marks loan start time）'pymnt_plan', # Payment plan'initial_list_status', # columns'application_type', # Application type'policy_code', # Policy code# 3. Credit history（Credit status before application）'dti', # Debt-to-income ratio'delinq_2yrs', # Delinquencies in past 2 years'earliest_cr_line', # Earliest credit line date'fico_range_low', # FICO score range low'fico_range_high', # FICO score range high'inq_last_6mths', # Inquiries in last 6 months'mths_since_last_delinq', # Months since last delinquency'mths_since_last_record', # Months since last record'open_acc', # Open credit lines'pub_rec', # Public derogatory records'revol_bal', # Revolving balance'revol_util', # Revolving utilization'total_acc', # Total credit lines'collections_12_mths_ex_med', # Collections in last 12 months（excluding medical）'mths_since_last_major_derog', #'acc_now_delinq', #'tot_coll_amt', #'tot_cur_bal', # all'open_acc_6m', # 6'open_act_il', #'open_il_12m', # 12'open_il_24m', # 24'mths_since_rcnt_il', #'total_bal_il', #'il_util', # Installment Use'open_rv_12m', # 12'open_rv_24m', # 24'max_bal_bc', # rows'all_util', # all Use'total_rev_hi_lim', #'inq_fi', #'total_cu_tl', #'inq_last_12m', # 12'acc_open_past_24mths',# 24'avg_cur_bal', # all'bc_open_to_buy', # rows'bc_util', # rows Use'chargeoff_within_12_mths', # 12'delinq_amnt', #'mo_sin_old_il_acct', #'mo_sin_old_rev_tl_op',#'mo_sin_rcnt_rev_tl_op',#'mo_sin_rcnt_tl', #'mort_acc', #'mths_since_recent_bc',# rows'mths_since_recent_bc_dlq', # rows'mths_since_recent_inq', #'mths_since_recent_revol_delinq', #'num_accts_ever_120_pd', # 120'num_actv_bc_tl', # rows'num_actv_rev_tl', #'num_bc_sats', # rows'num_bc_tl', # rows'num_il_tl', #'num_op_rev_tl', #'num_rev_accts', #'num_rev_tl_bal_gt_0', # >0'num_sats', #'num_tl_120dpd_2m', # 2 120'num_tl_30dpd', # 30'num_tl_90g_dpd_24m', # 24 90'num_tl_op_past_12m', # 12'pct_tl_nvr_dlq', #'percent_bc_gt_75', # Use >75% rows'pub_rec_bankruptcies',#'tax_liens', #'tot_hi_cred_lim', # all and'total_bal_ex_mort', #'total_bc_limit', # rows'total_il_high_credit_limit', ## 4. （Ifisjoint application）'annual_inc_joint', # Annual income'dti_joint', # Debt-to-income ratio'verification_status_joint', # Verification status'sec_app_fico_range_low', # FICO'sec_app_fico_range_high', # FICO'sec_app_earliest_cr_line', #'sec_app_inq_last_6mths', # 6'sec_app_mort_acc', #'sec_app_open_acc', #'sec_app_revol_util', # Use'sec_app_open_act_il', #'sec_app_num_rev_accts', #'sec_app_chargeoff_within_12_mths', # 12'sec_app_collections_12_mths_ex_med', # 12'sec_app_mths_since_last_major_derog', #]# ========================================================================# POST-LOAN FEATURES (Post-loan - Must delete, causes leakage)# ========================================================================post_loan_features = [# 1. Payment related（Post-loan ）'out_prncp', # Unpaid principal'out_prncp_inv', # Unpaid principal'total_pymnt', # Total payment received'total_pymnt_inv', # Total payment received'total_rec_prncp', # Principal received'total_rec_int', # Interest received'total_rec_late_fee', # Late fee received'recoveries', # Recoveries（After default）'collection_recovery_fee', # Collection fee'last_pymnt_d', # Last payment date'last_pymnt_amnt', # Last payment amount'next_pymnt_d', # Next payment date'last_credit_pull_d', # Last credit pull date'last_fico_range_high',# Recent FICO score high'last_fico_range_low', # Recent FICO score low# 2. Hardship/restructuring related（Post-loan ）'hardship_flag', # Hardship flag'hardship_type', # Hardship type'hardship_reason', # Hardship reason'hardship_status', # Hardship status'deferral_term', # Term'hardship_amount', # Hardship amount'hardship_start_date', # Hardship start date'hardship_end_date', # Hardship end date'payment_plan_start_date', # Payment planstart'hardship_length', # Hardship length'hardship_dpd', # Hardship DPD'hardship_loan_status',# Hardship loan status'orig_projected_additional_accrued_interest', # Original projected additional accrued interest'hardship_payoff_balance_amount', # Hardship payoff balance'hardship_last_payment_amount', # Hardship last payment amount# 3. Debt settlement related（After default ）'debt_settlement_flag', # Debt settlement flag'debt_settlement_flag_date', # Debt settlement date'settlement_status', # Settlement status'settlement_date', # Settlement date'settlement_amount', # Settlement amount'settlement_percentage', # Settlement percentage'settlement_term', # Term]# ========================================================================# METADATA/ID FEATURES (Metadata - Delete, no predictive value)# ========================================================================metadata_features = ['id', # Loan ID'member_id', # Member ID'url', # Loan URL'desc', # （ OCEAN ）'zip_code', # Zip code（Too fine-grained）'addr_state', # State（Potentially useful, but mark first）]# ========================================================================# TARGET VARIABLE# ========================================================================target_variable = 'loan_status'# ：desc ismetadata（ deleteID ）， feature （OCEAN ）# ：# 1. desc OCEAN feature# 2. OCEAN feature ，delete desc# 3. metadata_features in desc， atfeatureprint(f" PRE-LOAN features: {len(pre_loan_features)} ")print(f" POST-LOAN features: {len(post_loan_features)} ")print(f" METADATA features: {len(metadata_features)} ")print(f" Target variable: 1 ")print(f"\nTotal defined: {len(pre_loan_features) + len(post_loan_features) + len(metadata_features) + 1} ")

In [ ]:
# Cell 4: Check classification coverageprint("=" * 80)print("Check feature classification coverage")print("=" * 80)# datasetin allcolumnsall_columns_in_data = set(df.columns)# allcolumnsall_classified = set(pre_loan_features + post_loan_features + metadata_features + [target_variable])# columnsunclassified = all_columns_in_data - all_classified# in datain at columnsnot_in_data = all_classified - all_columns_in_dataprint(f"\ndatasetin columns : {len(all_columns_in_data)}")print(f" columns : {len(all_classified)}")if unclassified:print(f"\nWARNING: columns ({len(unclassified)} ):")for col in sorted(unclassified):print(f" - {col}")else:print(f"\n allcolumns ")if not_in_data:print(f"\nWARNING: in at datain at columns ({len(not_in_data)} ):")for col in sorted(not_in_data):# checkif col in pre_loan_features:category = "PRE-LOAN"elif col in post_loan_features:category = "POST-LOAN"elif col in metadata_features:category = "METADATA"else:category = "TARGET"print(f" - {col} ({category})")else:print(f"\n all columns at datain")

In [ ]:
# Cell 5: Create feature classification DataFrameprint("=" * 80)print("Create feature classification report")print("=" * 80)# Create classification dictionaryfeature_classification = {}# PRE-LOAN featuresfor feat in pre_loan_features:if feat in df.columns:coverage = (df[feat].notna().sum() / len(df)) * 100feature_classification[feat] = {'category': 'PRE-LOAN','usable': ' Yes','leakage_risk': ' No','coverage_%': f"{coverage:.2f}%",'dtype': str(df[feat].dtype),'note': 'Safe to use - known at application time'}# POST-LOAN featuresfor feat in post_loan_features:if feat in df.columns:coverage = (df[feat].notna().sum() / len(df)) * 100feature_classification[feat] = {'category': 'POST-LOAN','usable': ' No','leakage_risk': 'WARNING: HIGH','coverage_%': f"{coverage:.2f}%",'dtype': str(df[feat].dtype),'note': 'MUST DELETE - causes data leakage'}# METADATA featuresfor feat in metadata_features:if feat in df.columns:coverage = (df[feat].notna().sum() / len(df)) * 100note = 'Delete - no predictive value' if feat != 'desc' else 'Keep for OCEAN extraction, then delete'feature_classification[feat] = {'category': 'METADATA','usable': ' No' if feat != 'desc' else 'WARNING: Special','leakage_risk': ' No','coverage_%': f"{coverage:.2f}%",'dtype': str(df[feat].dtype),'note': note}# TARGET variableif target_variable in df.columns:coverage = (df[target_variable].notna().sum() / len(df)) * 100feature_classification[target_variable] = {'category': 'TARGET','usable': ' Target','leakage_risk': 'N/A','coverage_%': f"{coverage:.2f}%",'dtype': str(df[target_variable].dtype),'note': 'This is what we want to predict'}# create DataFrameclassification_df = pd.DataFrame.from_dict(feature_classification, orient='index')classification_df.index.name = 'feature_name'classification_df = classification_df.reset_index()# Categorycategory_order = {'PRE-LOAN': 1, 'POST-LOAN': 2, 'METADATA': 3, 'TARGET': 4}classification_df['sort_order'] = classification_df['category'].map(category_order)classification_df = classification_df.sort_values('sort_order').drop('sort_order', axis=1)print(f"\nFeature classification summary:")print(classification_df.groupby('category').size())print(f"\n 20rows :")display(classification_df.head(20))

In [ ]:
# Cell 6: Save feature classification lists print("=" * 80) print("Save feature classification lists") print("=" * 80) # 1. saveComplete classification output_file_1 = '../../feature_classification_complete.csv' classification_df.to_csv(output_file_1, index=False) print(f"\n Complete classification saved: {output_file_1}") # 2. save PRE-LOAN features （Can use ） pre_loan_df = classification_df[classification_df['category'] == 'PRE-LOAN'].copy() output_file_2 = '../../features_pre_loan.csv' pre_loan_df.to_csv(output_file_2, index=False) print(f" PRE-LOAN features save: {output_file_2}") print(f" - {len(pre_loan_df)} available features") # 3. save POST-LOAN features （Must delete ） post_loan_df = classification_df[classification_df['category'] == 'POST-LOAN'].copy() output_file_3 = '../../features_post_loan.csv' post_loan_df.to_csv(output_file_3, index=False) print(f" POST-LOAN features save: {output_file_3}") print(f" - {len(post_loan_df)} features to delete（causes leakage）") # 4. save METADATA features （ delete ） metadata_df = classification_df[classification_df['category'] == 'METADATA'].copy() output_file_4 = '../../features_metadata.csv' metadata_df.to_csv(output_file_4, index=False) print(f" METADATA features save: {output_file_4}") print(f" - {len(metadata_df)} features to delete（no predictive value）") # 5. create featurecolumns （ ） # Feature_Name pre_loan_list = pre_loan_df['feature_name'].tolist() post_loan_list = post_loan_df['feature_name'].tolist() metadata_list = metadata_df['feature_name'].tolist() # saveasPython with open('../../feature_lists.py', 'w') as f: f.write("# Feature Lists Generated by 02_feature_selection_and_leakage_check.ipynb\n\n") f.write("# PRE-LOAN Features (Safe to use)\n") f.write("PRE_LOAN_FEATURES = [\n") for feat in pre_loan_list: f.write(f" '{feat}',\n") f.write("]\n\n") f.write("# POST-LOAN Features (MUST DELETE - causes leakage)\n") f.write("POST_LOAN_FEATURES = [\n") for feat in post_loan_list: f.write(f" '{feat}',\n") f.write("]\n\n") f.write("# METADATA Features (DELETE - no predictive value)\n") f.write("METADATA_FEATURES = [\n") for feat in metadata_list: f.write(f" '{feat}',\n") f.write("]\n\n") f.write(f"# TARGET Variable\n") f.write(f"TARGET_VARIABLE = '{target_variable}'\n") print(f"\n Pythonfeaturecolumns save: feature_lists.py") print(f"\nall save！")

In [ ]:
# Cell 7: generatefinalsummary reportprint("=" * 80)print("Feature Classification & Data Leakage Check - Final Report")print("=" * 80)print(f"\n Dataset Information:")print(f" - rows : {len(df):,}")print(f" - columns : {len(df.columns)}")print(f"\n Feature Classification Results:")print(f" PRE-LOAN features: {len(pre_loan_df)} ")print(f" - Safe to use - known at application time modeltrain")print(f" - Information known at loan application time")print(f"\n POST-LOAN features: {len(post_loan_df)} ")print(f" - Must delete！Causes data leakage")print(f" - Information generated after loan issuance")print(f" - Using these features causes overfitting")print(f"\n METADATA features: {len(metadata_df)} ")print(f" - Recommend delete (except desc for OCEAN)")print(f" - ID no predictive value")print(f"\n Target variable: 1 ({target_variable})")print(f"\nWARNING: Data Leakage Risk Analysis:")leakage_features = classification_df[classification_df['leakage_risk'] == 'WARNING: HIGH']print(f" - Found {len(leakage_features)} high-risk features")print(f" - These features unavailable in real application")print(f" - Must delete before modeling")print(f"\n Modeling Recommendations:")print(f" 1. Use {len(pre_loan_df)} PRE-LOAN features")print(f" 2. Extract OCEAN features from desc field (5)")print(f" 3. Expected final feature count: {len(pre_loan_df) + 5} ")print(f" 4. Target variable: {target_variable}")print(f"\n Output files:")print(f" feature_classification_complete.csv - Complete classification")print(f" features_pre_loan.csv - available features ")print(f" features_post_loan.csv - Features to delete list（Leakage）")print(f" features_metadata.csv - Features to delete list（Metadata）")print(f" feature_lists.py - Pythonfeaturecolumns ")print(f"\n Next Steps:")print(f" 1. rows 03_create_modeling_dataset.ipynb")print(f" 2. delete POST-LOAN and METADATA features")print(f" 3. PRE-LOAN features + desc + target")print(f" 4. Create clean modeling dataset")print(f"\n" + "=" * 80)print(f"Feature classification completed！")print("=" * 80)

In [ ]:
# Cell 8: display Leakage feature Sample dataprint("=" * 80)print("Data Leakage Example Analysis")print("=" * 80)print("\n is POST-LOAN features， data leakage:\n")# leakagefeatureleakage_examples = ['total_pymnt','total_rec_prncp','total_rec_int','out_prncp','recoveries']# Filter existing featuresleakage_examples = [f for f in leakage_examples if f in df.columns]if leakage_examples and 'loan_status' in df.columns:# Show relationship between these features and loan_statussample_cols = ['loan_status'] + leakage_examplessample_df = df[sample_cols].head(10)print("\nSample data（ 10rows）:")display(sample_df)print("\nWARNING: Why these features cause Leakage?")print("\n1. total_pymnt (Total payment):")print(" - Only know total payment after loan ends")print(" - If loan defaults(Charged Off)，total_pymntwill be low")print(" - If loan fully paid(Fully Paid)，total_pymnt Loan amount+ ")print(" - Model will learn：total_pymnt ➜ Fully Paid，total_pymnt ➜ Charged Off")print(" - at ， atPre-loan total_pymnt！")print("\n2. out_prncp (Unpaid principal):")print(" - haveat rowsin ")print(" - If out_prncp = 0 ➜ definitely is Fully Paid")print(" - If out_prncp > 0 ➜ might be Charged Off or still in repayment")print("\n3. recoveries (Recoveries):")print(" - haveat After default， ")print(" - If recoveries > 0 ➜ definitely defaulted")print(" - isAfter default data！")print("\n Correct approach:")print(" Only use information known at application time：")print(" - annual_inc (Annual income)")print(" - dti (Debt-to-income ratio)")print(" - fico_range_low (FICO )")print(" - emp_length (Employment length)")print(" - purpose (Loan purpose)")print(" - etc.")else:print("\nWARNING: Example features not in data, skip display")